# Explorando conjuntos de datos con pandas y Matplotlib

El conjunto de datos que exploraremos contiene datos anuales sobre los flujos de migrantes internacionales registrados por los países de destino. Los datos presentan tanto las entradas como las salidas según el lugar de nacimiento, ciudadanía o lugar de residencia anterior/siguiente tanto para extranjeros como para nacionales.

Conjunto de datos: Inmigración a Canadá de 1980 a 2013 - Flujos migratorios internacionales hacia y desde países seleccionados - Revisión de 2015 del sitio web de las Naciones Unidas.

# Descarga y preparación de los datos

In [ ]:
# importar los módulos para el análisis de datos: pandas y numpy
import numpy as np  # para cálculos en Python
import pandas as pd # para estructura de datos

In [ ]:
# descarga y lectura de los datos
df_can = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Visualizacion de datos con Python/Canada.xlsx',
    sheet_name='Canada by Citizenship',
    skiprows=range(20),
    skipfooter=2)
print('Datos descargados y leídos con éxito!')

In [ ]:
# osbservamos los primeros cinco elementos
df_can.head()

In [ ]:
# los últimos cinco elementos
df_can.tail()

In [ ]:
# analizamos la dimensión del dataframe
print(df_can.shape)

In [ ]:
# breve resumen del dataframe
df_can.info(verbose=False)

In [ ]:
# información general del dataframe
print('Cantidad de Filas y columnas:', df_can.shape, '\n')
print('Nombre de las columnas:',df_can.columns, '\n')
print('Información sobre el dataset:', '\n')
df_can.info()

In [ ]:
# índices utilizados
df_can.index

In [ ]:
# buscar duplicados
df_can.duplicated().value_counts

In [ ]:
# identificar valores faltantes
df_can.isnull().values.any()

In [ ]:
print(df_can.isnull().sum())

In [ ]:
datos_faltantes = df_can.isnull()
datos_faltantes.head(5)

In [ ]:
# contar valores faltantes
for column in datos_faltantes.columns.values.tolist():
    print(column)
    print (datos_faltantes[column].value_counts())
    print("")

In [ ]:
# valores únicos para cada columna
df_can.nunique()

In [ ]:
# tipos de datos por columna
print(df_can.dtypes)

In [ ]:
# contar tipos de datos
numericos = df_can.select_dtypes(include=['int', 'float']).shape[1]
objetos = df_can.select_dtypes(include='object').shape[1]
print(f"Número de columnas numéricas: {numericos}")
print(f"Número de columnas de tipo objeto: {objetos}")

In [ ]:
print("\nResumen estadístico:")
print(df_can.describe())

In [ ]:
print("\nResumen estadístico categóricas:")
print(df_can.describe(include=['object']))

limpiamos el conjunto de datos para eliminar las columnas que consideramos inesesarias para el análisis

In [ ]:
# eliminamos las columnas
df_can.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=True)

# visualizamos nuevamente el daframe para ver los cambios
df_can.head()

In [ ]:
# cambiamos los nombres de las columnas para que queden mas y tengan mas sentido
df_can.rename(columns={'OdName':'Pais', 'AreaName':'Continente','RegName':'Region'}, inplace=True)
df_can.head()

Chequear que todas las etiquetas de las columnas sean de tipo string, para asegurar la coherencia

In [ ]:
# chequeamos los tipos de etiqueta
all(isinstance(column, str) for column in df_can.columns)

Como devolvió False vamos a cambiar todos a tipo string


In [ ]:
# cambiamos a tipo string
df_can.columns = list(map(str, df_can.columns))

# chequeamos los tipos de etiqueta nuevamente
all(isinstance(column, str) for column in df_can.columns)

Para buscar países de forma más rápida vamos a establecer la columna pais como índice

In [ ]:
# establecer país como índice
df_can.set_index('Pais', inplace=True)

# visualizamos
df_can.head()

Vamos a agregar una columna con el total de inmigrantes por cada país

In [ ]:
# agregamos la columna Total y visualizamos el dataframe nuevemente para ver el resultado
df_can['Total'] = df_can.sum(axis=1)

df_can.head()

In [ ]:
# visualizamos nuevamente la dimensión del dataframe, para chequear que tenga una columna más (de 37 a 38)
print('dimensión de los datos:', df_can.shape)

In [ ]:
# para poder realizar los gráficos, vamos a crear una lista de años (desde 1980 a 2013)
fechas = list(map(str, range(1980, 2014)))

fechas

# Visualización con Matplotlib

Importamos la librería

In [ ]:
# utilizamos backend online
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

# librería para los gráficos de gofres (waffle)
import matplotlib.patches as mpatches

mpl.style.use('ggplot')

# última versión de Matplotlib
print('Versión de Matplotlib: ', mpl.__version__)

## Gráficos de área

Este tipo de gráficos se utilizan para representar datos numéricos a lo largo del tiempo o en diferentes categorías. Muestran la acumulación de valores, permitiendo comparar tendencias de diferentes series de datos. Pueden ser apilados o superpuestos según para lo que fuere necesario

In [ ]:
# ordenamos por Total
df_can.sort_values(['Total'], ascending=False, axis=0, inplace=True)

# tomamos los 5 mejores
df_top5 = df_can.head()

# cambiamos el dataframe
df_top5 = df_top5[fechas].transpose()

df_top5.head()

Los gráficos de área se apilan de forma predeterminada. Y para producir un gráfico de áreas apiladas, cada columna debe tener todos valores positivos o todos negativos (cualquier valor NaN, tendrá el valor predeterminado 0). Para producir un gráfico desapilado, hay que establecer el parámetro stackeden en valor False.

Graficamos la tendencia de inmigración de los 5 principales países

In [ ]:
# cambiamos los valores del índice del dataframe a entero para graficar
df_top5.index = df_top5.index.map(int)

# graficamos
df_top5.plot(kind='area',
            alpha=0.25,  # valor de la transparencia
            stacked=False,
            figsize=(20, 10))

# agregamos títulos
plt.title('Inmigración de los 5 principales países')
plt.ylabel('Número de Inmigrantes')
plt.xlabel('Años')

# mostramos el gráfico
plt.show()

Graficamos los 5 países que menos contribuyeron a la inmigración a Canadá entre 1980 y 2013, con valor transparencia de 0,45.

In [ ]:
# definimos un nuevo df
df_least5 = df_can.tail(5)

# transponemos
df_least5 = df_least5[fechas].transpose()
df_least5.head()

# cambiamos los valores del índice a enteros
df_least5.index = df_least5.index.map(int)

# graficamos
df_least5.plot(kind='area',
               alpha=0.45,
               figsize=(20, 10))

# agregamos títulos
plt.title('Tendencia migratoria de los 5 países con menor contribución a la inmigración')
plt.ylabel('Número de Inmigrantes')
plt.xlabel('Años')

# mostramos el gráfico
plt.show()

Podemos graficar lo mismo utilizando una instancia de Matplotlib llamada Axes. Utilizamos esta instancia y la almacenamos en una variable, (por ejemplo ax), para luego poder agregar el resto de los elementos, ax.set_title().

In [ ]:
df_least5 = df_can.tail(5)

# transponemos
df_least5 = df_least5[fechas].transpose()

df_least5.head()

# cambiamos los valores del índice a enteros
df_least5.index = df_least5.index.map(int)

# graficamos
ax = df_least5.plot(kind='area',
                    alpha=0.55,
                    stacked=False, # para que no quede apilado
                    figsize=(20, 10))

# agregamos los títulos
ax.set_title('Tendencia migratoria de los 5 países con menor contribución a la inmigración')
ax.set_ylabel('Número de Inmigrantes')
ax.set_xlabel('Años')


## Histogramas

Los histogramas representan la distribución de datos numéricos continuos en intervalos discretos llamados "bins" (contenedores). Cada bin representa un rango de valores y la altura de la barra indica la frecuencia de datos en ese rango. Este tipo de gáraficos son útiles para comprender la distribución de datos y detectar patrones.
Generlamente es necesario modificar el tamaño del contenedor para que la distribución se muestre bien.

A continuación queremos saber: ¿Cuál es la distribución de frecuencia del número (población) de nuevos inmigrantes a Canadá en 2013 de los distintos países?

In [ ]:
# visualizamos los datos de 2013
df_can['2013'].head()

In [ ]:
# np.histogram nos devuelve 2 valores
conteo_frec, rango_bin = np.histogram(df_can['2013'])

print(conteo_frec) # conteo de la frecuencia
print(rango_bin) # rango de contenedores, por defecto 10 bins

Su lectura sería la siguiente:
- 178 países aportaron entre 0 y 3412,9 inmigrantes.
- 11 países aportaron entre 3.412,9 y 6.825,8 inmigrantes
- 1 país aportó entre 6285,8 y 10238,7 inmigrantes, y así sucesivamente.

In [ ]:
# graficcamos utilizando kind='hist'
# agregamos xticks para que las etiquetas del eje x coincidan con el tamaño del contenedor
df_can['2013'].plot(kind='hist', figsize=(8, 5), xticks=rango_bin)

# agregamos un título
plt.title('Histograma de inmigración de 195 países en 2013')

# agregamos etiqueta en eje y
plt.ylabel('Número de países')

# agregamos etiqueta en eje x
plt.xlabel('Número de inmigrantes')

# mostramos el gráfico
plt.show()

Ahora queremos saber: Cuál es la distribución de la inmigración para China, India y Philippines durante los años 1980 - 2013?

In [ ]:
# veamos primero como esta nuestro df
df_can.head()

In [ ]:
# visualizamos el df con los países seleccionados
df_can.loc[['China', 'India', 'Philippines'], fechas]

In [ ]:
# transponemos el df para que el gráfico represente los datos de forma correcta
df_cip = df_can.loc[['China', 'India', 'Philippines'], fechas].transpose()
df_cip.head()

In [ ]:
# obtenemos los valores
conteo_frec, rango_bin = np.histogram(df_cip, 15)

# graficamos (histograma)
df_cip.plot(kind ='hist',
          figsize=(10, 6),
          bins=15,
          alpha=0.35,
          xticks=rango_bin,
          color=['coral', 'darkslateblue', 'mediumseagreen'],
          stacked=True, # para que no se superpongan
         )

# agregamos título y etiquetas
plt.title('Histograma de inmigración de China, India y Philippines de 1980 a 2013')
plt.ylabel('Número de Años')
plt.xlabel('Número de inmigrantes')

plt.show()

## Gráficos de barras

Los gráficos de barras son ideales para datos categóricos y compararlos. Cada barra representa una categoría y su longitud es proporcional al valor que representa. Los gráficos de barras se pueden utilizar en orientación vertical u horizontal.
Este tipo de gráficos suelen representar variables numéricas y categóricas agrupadas en intervalos.

*   `kind=bar` crea un diagrama de barras verticales
*   `kind=barh` crea un diagrama de barras horizontales

In [ ]:
# tomamos los datos que queremos graficar
df_denmark = df_can.loc['Denmark', fechas]
df_denmark.head()

In [ ]:
# trazamos un gráfico de barras verticales
df_denmark.plot(kind='bar', figsize=(10, 6))

# agregamos títulos y etiquetas
plt.xlabel('Años')
plt.ylabel('Número de inmigrantes')
plt.title('Inmigrantes de Dinamarka a Canadá de 1980 a 2013')

# mostramos el gráfico
plt.show()

In [ ]:
# trazamos un gráfico de barras horizontal
# que muestre el número total de inmigrantes a Canadá desde los 15 países principales, para el período 1980 - 2013

# ordenamos el df por la columan total de forma ascendente
df_can.sort_values(by='Total', ascending=True, inplace=True)

# obtenemos el df para graficar con los 15 países principales
df_top15 = df_can['Total'].tail(15)
df_top15

In [ ]:
# graficamos
df_top15.plot(kind='barh', figsize=(15, 11), color='darkseagreen')

# agregamos títulos y etiquetas
plt.xlabel('Número de inmigrantes')
plt.ylabel('País')
plt.title('Los 15 países que más contribuyeron a la inmigración a Canadá entre 1980 y 2013')

# agregamos etiquetas de valor a cada país
for index, value in enumerate(df_top15):
    label = format(int(value), ',')

# texto al final de la barra
plt.annotate(label, xy=(value - 57000, index - 0.10), color='white')

plt.show()

## Gráficos circulares

Este tipo de gráfico es ideal para representar partes de un todo. Son adecuados para mostrar proporciones y distribuciones, pero deben utilizarse con precaución para evitar confusiones.

*   `kind=pie` crea un gráfico circular

Queremos explorar la proporción (porcentaje) de nuevos inmigrantes agrupados por continentes durante todo el período de 1980 a 2013.

In [ ]:
# visualizamos nuestro df para ver su estado
df_can.head()

In [ ]:
# primero agrupamos los países por continentes y aplicamos la función suma()
df_continentes = df_can.groupby('Continente', axis=0).sum()

print(type(df_can.groupby('Continente', axis=0)))

df_continentes.head()

In [ ]:
# definimos los colores y el ratio que vamos a utilizar para el gráfico
lista_colores = ['tan', 'yellowgreen', 'lightcoral', 'turquoise', 'sandybrown', 'pink']
lista_ratio = [0.1, 0, 0, 0, 0.1, 0.1]

# luego trazamos el gráfico
df_continentes['Total'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%',   # formato del valor de las etiquetas
                            startangle=90,
                            shadow=True,
                            labels=None,         # desactivamos las etiquetas del gráfico
                            pctdistance=1.15,    # distancia entre el centro de cada sector circular y el inicio del texto
                            colors=lista_colores,  # los colores dela lista personalizada
                            explode=lista_ratio # "resaltar" los 3 continentes más bajos
                            )

# ampliamos el título en un 15 % e igualamos distancia
plt.title('Inmigración a Canadá por continente entre 1980 y 2013', y=1.15)

plt.axis('equal')

# agregamos una leyenda
plt.legend(labels=df_continentes.index, loc='upper left')

plt.show()

Podemos realizar otras exploraciones, como el porcentaje de nuevos inmigrantes agrupados por continentes en el año 2010.

In [ ]:
# definimos otro ratio y troso colores
lista_ratio = [0.0, 0, 0, 0.1, 0.1, 0.2]
lista_colores = ['tan', 'yellowgreen', 'lightcoral', 'turquoise', 'sandybrown', 'pink']

# trazamos el gráfico
df_continentes['2010'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%',
                            startangle=90,
                            shadow=True,
                            labels=None,
                            pctdistance=1.12,
                            colors=lista_colores,
                            explode=lista_ratio
                            )

# ampliamos el título en un 15 % e igualamos distancia
plt.title('Inmigración a Canadá por continente en 2008', y=1.15)

plt.axis('equal')

# agregamos una leyenda
plt.legend(labels=df_continentes.index, loc='upper left')

plt.show()

## Gráficos de cajas

Ese tipo de gráfico es útil para visualizar la distribución y los valores atípicos de un conjunto de datos. Son ideales para comparar muúltiples distribuciones.

*   `kind=box` crea un gráfico de caja

- Mínimo: el número más pequeño del conjunto de datos, excluyendo los valores atípicos.
- Primer cuartil: número medio entre el mínimo y la mediana.
- Segundo cuartil (Mediana): número medio del conjunto de datos (ordenado).
- Tercer cuartil: Número medio entre la mediana y máximo.
- Máximo: el número más grande en el conjunto de datos, excluyendo los valores atípicos.

In [ ]:
# obtenemos el conjunto de datos, por ejemplo los relacionados a Japón
df_japon = df_can.loc[['Japan'], fechas].transpose()
df_japon.head()

In [ ]:
# trazamos el grafico
df_japon.plot(kind='box', figsize=(8, 6))

plt.title('Diagrama de caja de inmigrantes japoneses de 1980 a 2013')
plt.ylabel('Número de inmigrantes')

plt.show()

A simple vista podemos decir que:
- El número mínimo de inmigrantes es de alrededor de 200, el número máximo es de alrededor de 1300 y el número medio de inmigrantes es de alrededor de 900 (mediana).
- El 25% de los años del período 1980 - 2013 tuvieron un recuento anual de inmigrantes de ~500 o menos (primer cuartil).
- El 75% de los años del período 1980 - 2013 tuvieron un recuento anual de inmigrantes de ~1100 o menos (tercer cuartil).

In [ ]:
# verificamos
df_japon.describe()

Comparemos ahora la distribución del número de nuevos inmigrantes de India y China para el período 1980 - 2013.

In [ ]:
# obtenemos el conjunto de datos China e India
df_CI= df_can.loc[['China', 'India'], fechas].transpose()
df_CI.head()

In [ ]:
# graficamos
df_CI.plot(kind='box', figsize=(10, 7))

plt.title('Diagramas de caja de inmigrantes de China e India (1980 - 2013)')
plt.ylabel('Número de inmigrantes')

plt.show()

In [ ]:
# podemos graficarlo de forma horizontal también, pasando el parámetro vert en False
df_CI.plot(kind='box', figsize=(10, 7), color='blue', vert=False)

plt.title('Diagramas de caja de inmigrantes de China e India (1980 - 2013)')
plt.ylabel('Número de inmigrantes')

plt.show()

In [ ]:
# verificamos
df_CI.describe()

Ahora podemos observar que, si bien ambos países tienen aproximadamente la misma población de inmigrantes - media (~20.000), el rango de población inmigrante de China está más extendido que el de la India. La población máxima de la India es de (36.210) que es aproximadamente un 15% menor que la población máxima de China (42.584).

## Gráficos de dispersión

Se considera que los datos de un diagrama de dispersión expresan una tendencia. Es un método útil para comparar variables entre sí. Con un análisis más detallado que utilice herramientas como la regresión, podemos calcular matemáticamente esta relación y usarla para predecir tendencias fuera del conjunto de datos.

*   `kind=scatter` crea un gráfico de dipsersión

A continuación queremos visualizar la tendencia de la inmigración total a Canadá (todos los países combinados) para los años 1980 - 2013.

Utilizaremos para ello `scatter plot`

In [ ]:
# visualizamos nuestro df para ver su estado
df_can.head()

In [ ]:
# obtenemos el conjunto de datos.
# como vamos a utilizar la relación entre fechas (años) y total de población, convertimos fechas (años) en tipo int.

# utilizamos sum() para obtener la población total por año
df_total = pd.DataFrame(df_can[fechas].sum(axis=0))

# cambiamos fechas (años) a tipo int
df_total.index = map(int, df_total.index)

# reestablecemos el índice para volver a colocarlo como columna en el df
df_total.reset_index(inplace = True)

# renombramos las columnas
df_total.columns = ['año', 'total']

# visualizamos como queda el df
df_total.head()

In [ ]:
# graficamos agregando una línea de regresión para predecir el número de inmigrantes en 2015
# primero trazamos la línea de ajuste
x = df_total['año']
y = df_total['total']
fit = np.polyfit(x, y, deg=1)

print(fit)

# graficamos
df_total.plot(kind='scatter', x='año', y='total', figsize=(10, 6), color='darkblue')

plt.title('Total de inmigración a Canadá desde 1980 a 2013')
plt.xlabel('Años')
plt.ylabel('Número de Inmigrantes')

# inlcuimos al gráfico la línea de mejor ajuste
plt.plot(x, fit[0] * x + fit[1], color='red')
plt.annotate('y={0:.0f} x + {1:.0f}'.format(fit[0], fit[1]), xy=(2000, 150000))

plt.show()

# imprimimos la línea de mejor ajuste
print("\n")
'No. Inmigrantes = {0:.0f} * Año + {1:.0f}'.format(fit[0], fit[1])

Utilizando la ecuación de la línea de mejor ajuste: y = a * x + b, podemos predecir que para el año 2015 Canadá aceptará alrededor de 291.310 inmigrantes.

-No. Inmigrantes = 5567 * Año + -10926195

-No. Inmigrantes = 5567 * 2015 -10926195

-No. Inmigrantes = 291.310

Resumen de pasos realizados:
1. Obtuvimos el conjunto de datos necesario para visualizar la tendencia de la inmigración total a Canadá.
2. Utilizamos sum() para obtener la población total por año.
3. Lo nombramos df_total
4. Cambiamos fechas (años) a tipo int, ya que vamos a utilizar la relación entre fechas (años) y total de población.
5. Rreestablecemos el índice para volver a colocarlo como columna en el df
6. Renombramos las columnas a año y total.
7. Visualizamos como queda el df
8. Graficamos


## Gráficos de burbujas

Este tipo de gráfico muestra tres dimensiones de datos (x, y, z). Los puntos de datos se reemplazan por burbujas y el tamaño de la burbuja está determinado por la tercera variable z (variable de peso)

Analizaremos el efecto de la crisis Argentina.

Argentina sufrió una gran crisis entre 1998 y 2002, que provocó un desempleo generalizado, disturbios, la caída del gobierno y un impago de la deuda externa del país. En términos de ingresos, más del 50% de los argentinos eran pobres, y siete de cada diez niños argentinos eran pobres en el peor momento de la crisis de 2002.

Analicemos el efecto de esta crisis y comparemos la inmigración de Argentina con la de su vecino Brasil. Hagámoslo usando un cálculo bubble plotde la inmigración de Brasil y Argentina para los años 1980 - 2013. Estableceremos las ponderaciones de la burbuja como el valor normalizado de la población para cada año.

In [ ]:
# visualizamos nuestro df para ver su estado
df_can.head()

In [ ]:
# en primer lugar obtenemos los datos de Brasil y Argentina
# transponemos el df
df_can_t = df_can[fechas].transpose()

# ambiamos fechas (años) a tipo int
df_can_t.index = map(int, df_can_t.index)

# Etiquetemos el índice. Este será automáticamente el nombre de la columna cuando restablezcamos el índice.
df_can_t.index.name = 'Año'

# restablecer el índice para incluir el año como una columna
df_can_t.reset_index(inplace=True)

# visualizo el df para ver los cambios
df_can_t.head()

In [ ]:
# a continuación normalizamos los datos de Brasil y Argentina
# brasil
norm_brasil = (df_can_t['Brazil'] - df_can_t['Brazil'].min()) / (df_can_t['Brazil'].max() - df_can_t['Brazil'].min())

# argetina
norm_argentina = (df_can_t['Argentina'] - df_can_t['Argentina'].min()) / (df_can_t['Argentina'].max() - df_can_t['Argentina'].min())

Para trazar dos diagramas de dispersión diferentes en un diagrama, podemos incluir los ejes de un diagrama en el otro pasándolo a través del parámetro ax.

También pasaremos los pesos usando el parámetro s.

Dado que los pesos normalizados están entre 0 y 1, no serán visibles en el gráfico. Por lo tanto, haremos lo siguiente:
- multiplicamos los pesos por 2000 para ampliarlos en el gráfico
- agregamos 10 para compensar el valor mínimo (que tiene un peso de 0 y, por lo tanto, escala con $\times 2000$).

In [ ]:
# graficamos
# Brasil
ax0 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='Brazil',
                    figsize=(14, 8),
                    alpha=0.5,
                    color='green',
                    s=norm_brasil * 2000 + 10,
                    xlim=(1975, 2015)
                    )

# Argentina
ax1 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='Argentina',
                    alpha=0.5,
                    color="blue",
                    s=norm_argentina * 2000 + 10,
                    ax=ax0
                    )

ax0.set_ylabel('Número de inmigrantes')
ax0.set_title('Inmigración de Brasil y Argentina de 1980 a 2013')
ax0.legend(['Brasil', 'Argentina'], loc='upper left', fontsize='x-large')

El tamaño de la burbuja corresponde a la magnitud de la población inmigrante para ese año, en comparación con los datos de 1980 – 2013. Cuanto más grande es la burbuja, más inmigrantes habrá ese año.

En el gráfico anterior, podemos ver un aumento correspondiente en la inmigración desde Argentina durante la gran depresión de 1998 a 2002. También podemos observar un pico similar entre 1985 y 1993. De hecho, Argentina había sufrido una gran crisis entre 1974 y 1990, justo antes del inicio de la crisis de 1998 a 2002.

En una nota similar, Brasil sufrió el efecto Samba , donde el real brasileño (moneda) cayó casi un 35% en 1999. Existía el temor de una crisis financiera en América del Sur, ya que muchos países sudamericanos dependían en gran medida de las exportaciones industriales de Brasil. Posteriormente, el gobierno brasileño adoptó un programa de austeridad y la economía se recuperó lentamente a lo largo de los años, culminando con un aumento en 2010. Los datos de inmigración reflejan estos eventos.

Haremos algo similar con China e India para visualizar las diferencias de inmigración en el tiempo desde 1980 hasta 2013.
Utilizaremos el marcos  de datos df_can_t, del gráfico anterior-

In [ ]:
# visualizamos el df para su estado
df_can_t.head()

In [ ]:
# normalizamos los datos de China
norm_china = (df_can_t['China'] - df_can_t['China'].min()) / (df_can_t['China'].max() - df_can_t['China'].min())

# normalizamos los datos de India
norm_india = (df_can_t['India'] - df_can_t['India'].min()) / (df_can_t['India'].max() - df_can_t['India'].min())

In [ ]:
# graficamos
# China
ax0 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='China',
                    figsize=(14, 8),
                    alpha=0.5,  # transparencia
                    color='green',
                    s=norm_china * 2000 + 10,
                    xlim=(1975, 2015)
                    )

# India
ax1 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='India',
                    alpha=0.5,
                    color="blue",
                    s=norm_india * 2000 + 10,
                    ax = ax0
                    )

ax0.set_ylabel('Número de inmigrantes')
ax0.set_title('Inmigración de China e India de 1980 a 2013')
ax0.legend(['China', 'India'], loc='upper left', fontsize='x-large')

## Gráficos de gofres

Este tipo de gráfico normalmente se utiliza para mostrar el progreso hacia metas.

Suele ser una opción eficaz cuando se intenta agregar funciones de visualización interesantes a un objeto visual que consta principalmente de celdas, como un panel de Excel, (mosaicos).

A continuación vamos a revisar el caso de inmigración sobre Dinamarca, Noruega y Suecia.

In [ ]:
# visualizamos nuestro df para ver su estado
df_can.head()

In [ ]:
# creamos un nuevo df para estos tres países
df_dsn = df_can.loc[['Denmark', 'Norway', 'Sweden'], :]

# visualizamos el nuevo df
df_dsn

Como este tipo de gráficos no están integrados en ninguna de las bibliotecas de visualización de Python, tendremos que crearlos desde cero.

Resumen de pasos:
1. Determinar la proporción de cada categoría con respecto al total.
2. Definir el tamaño total del gráfico de waffle (gofre).
3. Usar la proporción de cada categoría para determinar su número respectivo de mosaicos.
4. Crear una matriz que se parezca al gráfico de waffle y completarla.
5. Asignar la matriz waffle del gráfico a un objeto visual.
6. Darle formato al gráfico

In [ ]:
# calculamos la proporción de cada categoría con respecto al total
valores_totales = df_dsn['Total'].sum()
proporciones_categoria = df_dsn['Total'] / valores_totales

# visualizamos proporciones
pd.DataFrame({"Proporción de categoría": proporciones_categoria})

In [ ]:
# definimos el tamaño total del gráfico de waffle
ancho = 40 # ancho del gráfico
alto = 10 # altura del gráfico

# número total de mosaicos
num_mosaic = ancho * alto

print(f'El número total de mosaicos es {num_mosaic}.')

In [ ]:
# calculamos el número de mosaicos para cada categoría
mosaicos_por_categoría = (proporciones_categoria * num_mosaic).round().astype(int)

# número de mosaicos por categoría
pd.DataFrame({"Número de mosaicos": mosaicos_por_categoría})

In [ ]:
# creamos una matriz que se parezca al gráfico de waffle y la completamos
grafico_waffle = np.zeros((alto, ancho), dtype = np.uint)

# definimos índices para recorrer el gráfico
category_index = 0
tile_index = 0

# rellenamos el gráfico
for col in range(ancho):
    for row in range(alto):
        tile_index += 1

        # si el número de mosaicos poblados para la categoría actual es igual a sus mosaicos asignados correspondientes...
        if tile_index > sum(mosaicos_por_categoría[0:category_index]):
            # ...pasar a la siguiente categoría
            category_index += 1

        # establezca el valor de la clase en un número entero, que aumenta con la clase
        grafico_waffle[row, col] = category_index

print ('Gráfico de Waffle completo!')

In [ ]:
# visualizamos la matríz
grafico_waffle

In [ ]:
# graficamos
# crear una instancia de un nuevo objeto de figura
fig = plt.figure()

# ustilizamos matshow para mostrar el gráfico
colormap = plt.cm.coolwarm
plt.matshow(grafico_waffle, cmap=colormap)
plt.colorbar()
plt.show()

In [ ]:
# le damos un poco de formato al gráfico
# crear una instancia de un nuevo objeto de figura
fig = plt.figure()

# ustilizamos matshow para mostrar el gráfico
colormap = plt.cm.coolwarm
plt.matshow(grafico_waffle, cmap=colormap)
plt.colorbar()

# eje
ax = plt.gca()

# divisiones
ax.set_xticks(np.arange(-.5, (ancho), 1), minor=True)
ax.set_yticks(np.arange(-.5, (alto), 1), minor=True)

# agregamos líneas de cuadrícula basadas en las divisiones
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

plt.xticks([])
plt.yticks([])

# calculamos la suma acumulada de categorías individuales para hacer coincidir los colores entre el gráfico y la leyenda
valores_acum = np.cumsum(df_dsn['Total'])
valores_totales = valores_acum[len(valores_acum) - 1]

# creamos la leyenda
leyendas = []
for i, category in enumerate(df_dsn.index.values):
    etiqueta = category + ' (' + str(df_dsn['Total'][i]) + ')'
    colores = colormap(float(valores_acum[i])/valores_totales)
    leyendas.append(mpatches.Patch(color=colores, label=etiqueta))

# agregamos leyenda al gráfico
plt.legend(handles=leyendas,
           loc='lower center',
           ncol=len(df_dsn.index.values),
           bbox_to_anchor=(0., -0.2, 0.95, .1)
          )
plt.show()

## Nubes de palabras

Las nubes se utilizan comúnmente para realizar análisis y visualización de alto nivel de datos de texto.

In [ ]:
# instalar wordcloud
!pip install wordcloud

# importar el paquete
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud se ha instalado e importado!')

Para esta ocasión analizaremos un cuento del escritor y aviador francés Antoine de Saint-Exupéry, titulado El Principito. Como primer paso descargar el archivo .txt del cuento.

In [ ]:
# importamos la librería si fuera necesario para abrir un archivo en una direción url
import urllib

el_principito = open('/content/drive/MyDrive/Colab Notebooks/Visualizacion de datos con Python/principito.txt').read()

A continuación, usemos las palabras vacías que importamos de word_cloud. Usamos el conjunto de funciones para eliminar cualquier palabra irrelevante redundante.

In [ ]:
stopwords = set(STOPWORDS)

Crear un objeto de nube de palabras y generar una nube de palabras. Para simplificar, generar una nube de palabras usando solo las primeras 2000 palabras del cuento.

In [ ]:
# crear una instancia de un objeto de nube de palabras
principito_wc = WordCloud(
     background_color='white',
     max_words=2000,
     stopwords=stopwords
 )

# generar la nube de palabras
principito_wc.generate(el_principito)

Una vez que está creada el siguiente paso es visualizarla.

In [ ]:
# visualizar la nube de palabras
plt.imshow(principito_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

En las primeras 2000 palabras del cuento, las palabras más comunes son lo , que, de, la, y, etc. Cambiar el tamaño de la nube para que podamos ver un poco mejor las palabras menos frecuentes.

In [ ]:
fig = plt.figure(figsize=(10, 12))

# visualizar la nube de palabras
plt.imshow(principito_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Como las palabras, que, la, y, de,lo y un, etc, no son palabras informativas, las agregamos a nuestras palabras vacías y volvemos a generar la nube. Aprovechamos y cambiamos el color y ampliamos el máximo de palabras

In [ ]:
principito_wc = WordCloud(
     background_color='white',
     max_words=3000,
     stopwords=["y", "de", "lo", "la", "que", "un", "una", "no", "a", "las", "lo", "en", "los", "en", "al", "el", "del", "mi", "para", "pero"],
     colormap = "magma")

# regenerar la nube de palabras
principito_wc.generate(el_principito)

# visualizar la nube
fig = plt.figure(figsize=(10, 12))

plt.imshow(principito_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Podemos darle forma a nuestra nube de palabras utlizando una máscara, por ejemplo para este caso forma de círculo

In [ ]:
# crear la máscara cículo
x, y = np.ogrid[:300, :300]
circulo = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
circulo = 255 * circulo.astype(int)

# crear una instancia de un objeto de nube de palabras y le damos formato
principito_wc = WordCloud(
     background_color='white',
     max_words=3000,
     stopwords=["y", "de", "lo", "la", "que", "un", "una", "no", "a", "las", "lo", "en", "los", "en", "al", "el", "del", "mi", "para", "pero"],
     colormap = "magma",
     repeat = True,
     mask = circulo)

# regenerar la nube de palabras
principito_wc.generate(el_principito)

# visualizar la nube
fig = plt.figure(figsize=(6, 8))

plt.imshow(principito_wc, interpolation='bilinear')
plt.axis('off')
plt.savefig("principito.png")
plt.show()

También podemos utilizar una imagen como mácara para dar forma a nuestra nube de palabras

In [ ]:
# tomar una imagen y guardar máscara como principito_mask
from PIL import Image
principito_mask = np.array(Image.open('/content/drive/MyDrive/Colab Notebooks/Visualizacion de datos con Python/principito.png'))

print('Imagen descargada y guardada!')

In [ ]:
# crear una instancia de un objeto de nube de palabras y le damos formato
principito_wc = WordCloud(
     background_color='white',
     max_words= 1000,
     stopwords=["y", "de", "lo", "la", "que", "un", "una", "no", "a", "las", "lo", "en", "los", "en", "al", "el", "del", "mi", "para", "pero"],
     colormap = "magma",
     repeat = True,
     mask = principito_mask)

# regenerar la nube de palabras
principito_wc.generate(el_principito)

# visualizar la nube
fig = plt.figure(figsize=(6, 8))

plt.imshow(principito_wc, interpolation='bilinear')
plt.axis('off')
plt.savefig("principito.png")
plt.show()

## Gráficos de regresión

Seaborn es una biblioteca de visualización de Python basada en matplotlib. Proporciona una interfaz de alto nivel para dibujar gráficos estadísticos atractivos. Exploraremos seaborn y veremos cuán eficiente es crear líneas de regresión y ajustes usando esta biblioteca.

In [ ]:
# instalar seaborn
!pip3 install seaborn

# importar la librería
import seaborn as sns

print('Seaborn instalada e importada!')

Para en este caso retomaremos el dataframe de Canadá para crear un nuevo marco de datos que almacene el número total de inmigrantes que llegaron a Canadá por año desde 1980 hasta 2013.

In [ ]:
# utilizar el método sum() para obtener la población total por año
df_total = pd.DataFrame(df_can[fechas].sum(axis=0))

# cambiar los años a float (útil para la regresión más adelante)
df_total.index = map(float, df_total.index)

# restablecer el índice para volver a colocarlo como una columna en el marco de datos df_total
df_total.reset_index(inplace=True)

# renombrar las columnas
df_total.columns = ['año', 'total']

# visualizar como queda el df final
df_total.head()

Generar el gráfico de regresión con Seaborn llamando a la función regplot.

In [ ]:
sns.regplot(x='año', y='total', data=df_total, color='green')

Damos un poco mas de formato al gráfico

In [ ]:
# tamaño del gráfico
plt.figure(figsize=(10, 5))

# tamaño de fuente de las etiquetas de las marcas de verificación, el título y las etiquetas x e y
sns.set(font_scale=1.2)

# cambiar el fondo a fondo con cuadrículas
sns.set_style('whitegrid')

# graficar
ax = sns.regplot(x='año', y='total', data=df_total, color='green', marker='+', scatter_kws={'s': 200})

# agregar etiquetas x e y
ax.set(xlabel='año', ylabel='Total Inmigración')

# # agregar título
ax.set_title('Total Immigración a Canada entre 1980 - 2013')

# mostrar gráfico
plt.show()

## Gráficos geoespaciales

Utilizaremos para este tipo de gráficos la biblioteca Folium.  Esta fue desarrollada con el único propósito de visualizar datos geoespaciales y el hecho de que los resultados de Folium sean interactivos hace que esta biblioteca sea muy útil para la creación de paneles.

Folium no está disponible de forma predeterminada. Por lo tanto, primero debemos instalarlo antes de poder importarlo.

In [ ]:
#!pip3 install folium==0.5.0
import folium

print('Folium instalado e importado!')

In [ ]:
# definimos un mapa mundial
mapa_mundo = folium.Map()

# mostrar mapa
mapa_mundo

Creamos un mapa centrado en Uruguay e implementamos niveles de zoom para ver cómo afecta el mapa renderizado.

In [ ]:
# definir el mapa mundial centrado alrededor de Uruguay con un nivel de zoom adecuado

# definir las coordenadas de Uruguay
uruguay_latitude = -32.522779
uruguay_longitude = -55.765835

# definir el mapa mundial centrado alrededor de Uruguay con un nivel de zoom adecuado
mapa_uruguay = folium.Map(location=[uruguay_latitude, uruguay_longitude], zoom_start=7)

# mostrar mapa
mapa_uruguay

Con Folium se pueden crear mapas con diferentes características. Probemos algunos

In [ ]:
# Mapas en blanco y negro (Stamen Toner Maps). Son perfectos para combinar datos y explorar meandros de ríos y zonas costeras.

# definir el mapa de Uruguay con un nivel de zoom adecuado y el estilo
mapa_uruguay = folium.Map(location=[uruguay_latitude, uruguay_longitude], zoom_start=7, tiles='Stamen Toner')

# mostrar mapa
mapa_uruguay

In [ ]:
# Mapas de terreno. Son mapas que presentan sombras de colinas y colores de vegetación natural.
# Muestran etiquetado avanzado y generalización de líneas de carreteras de doble calzada.

# definir el mapa de Uruguay con un nivel de zoom adecuado y el estilo
mapa_uruguay = folium.Map(location=[uruguay_latitude, uruguay_longitude], zoom_start=7, tiles='Stamen Terrain') # cartodb positron # OpenStreetMap

# mostrar mapa
mapa_uruguay

In [ ]:
# mapa de rutas

# definir el mapa de Uruguay con un nivel de zoom adecuado y el estilo
mapa_uruguay = folium.Map(location=[uruguay_latitude, uruguay_longitude], zoom_start=7, tiles='OpenStreetMap')

# mostrar mapa
mapa_uruguay

In [ ]:
# definir el mapa de Uruguay con un nivel de zoom adecuado y el estilo
mapa_uruguay = folium.Map(location=[uruguay_latitude, uruguay_longitude], zoom_start=7, tiles='cartodb positron')

# mostrar mapa
mapa_uruguay

**Mapas Coropléticos (mapas temáticos)**

Este tipo de mapas proporciona una manera fácil de visualizar cómo varía una medición en un área geográfica o muestra el nivel de variabilidad dentro de una región.

Crearemos un mapa Choropleth de Uruguay que represente la cantidad de población por departamento.

In [ ]:
# Crear un mapa de Folium centrado en Uruguay
uruguay = folium.Map(location=[-32.800085, -56.026504], zoom_start=7)

# Datos de población por departamento en Uruguay
data = {
    'Departamento': ['Artigas', 'Canelones', 'Cerro Largo', 'Colonia', 'Durazno', 'Flores', 'Florida', 'Lavalleja', 'Maldonado', 'Montevideo', 'Paysandú', 'Río Negro', 'Rivera', 'Rocha', 'Salto', 'San José', 'Soriano', 'Tacuarembó', 'Treinta y Tres'],
    'Poblacion': [73377, 520173, 84698, 123203, 57084, 25050, 67047, 58815, 164298, 1318755, 113107, 54765, 103473, 68088, 124861, 108304, 82594, 90051, 48134]
}

# Crear un DataFrame de Pandas a partir de los datos
datos = pd.DataFrame(data)

# crear la escala de colores
escala = np.linspace(datos['Poblacion'].min(),
                     datos['Poblacion'].max(),
                     16, dtype=int)

# cambiar la matriz a una lista
escala = escala.tolist()

# Agregar el Choropleth al mapa
folium.Choropleth(
    geo_data='/content/drive/MyDrive/Colab Notebooks/Visualizacion de datos con Python/uruguay.geojson',  # Archivo GeoJSON de los departamentos
    name='choropleth',
    data=datos,
    columns=['Departamento', 'Poblacion'],
    key_on='feature.properties.NAME_1',
    threshold_scale=escala,
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Población por Departamento en Uruguay'
).add_to(uruguay)

# visualizar mapa
uruguay


Algunos detalles de los parámetros:
1. **geo_data**: es el archivo GeoJSON.
2. **data**: es el marco de datos que contiene los datos.
3. **columns**: representa las columnas del marco de datos que se utilizarán para crear el Choroplethmapa.
4. **key_on**: es la clave o variable en el archivo GeoJSON que contiene el nombre de la variable de interés.
Para determinar eso, se deberá abrir el archivo GeoJSON usando cualquier editor de texto y anotar el nombre de la clave o variable que contiene el nombre de los departamentos, ya que los departamentos son nuestra variable de interés.
5. **threshold_scale**: escala.

**Mapas con Marcadores**

Para este tipo de mapas utilizaremos la siguiente información:


El Departamento de Policía de Gainesville (GPD) proporciona Crime Responses para documentar los detalles iniciales sobre un incidente al que responden los agentes de GPD.
Este conjunto de datos contiene incidentes delictivos desde 2011 hasta el presente e incluye un conjunto reducido de campos centrados en capturar el tipo de incidente, así como cuándo y dónde ocurrió.

Ref: https://data.cityofgainesville.org/Public-Safety/Crime-Responses/gvua-xt9q

In [ ]:
# descarga y lectura de los datos
df_incidentes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Visualizacion de datos con Python/Crime_Responses_20231016.csv')

print('Datos descargados y leídos con éxito!')

In [ ]:
df_incidentes.head()

Visualicemos cuántas entradas hay en el conjunto de datos.

In [ ]:
df_incidentes.shape

Para este caso vamos a reducir un poco los datos para trabajar mejor

In [ ]:
# obtener los primeros 100 delitos del df, para reducir un poco los datos
limit = 100
df_incidentes = df_incidentes.iloc[0:limit, :]

In [ ]:
df_incidentes.shape

In [ ]:
# cambiamos el nombre de la columna Incident Type
df_incidentes.rename(columns={'Incident Type':'Tipo'}, inplace=True)
df_incidentes.head(15)

Visualizaremos a continuación donde ocurrieron estos incidentes en la ciudad de Gainesville (Georgia).

In [ ]:
# variables para latitud y longitud de la ciudad de Gainesville (Georgia)
latitud_gainesville = 29.6516300
longitud_gainesville = -82.3248300

# crear el mapa y mostrarlo
mapa_gainesville = folium.Map(location=[latitud_gainesville, longitud_gainesville], zoom_start=12)

# visualizar el mapa
mapa_gainesville

Ahora superponemos las ubicaciones de los incidentes en el mapa. Agregamos texto emergente que se mostrará cuando pase el cursor sobre un marcador y hacemos que cada marcador muestre el tipo de incidente cuando pase el cursor sobre él.

In [ ]:
# crear una instancia de un grupo de características para los incidentes en el df
incidentes = folium.map.FeatureGroup()

# recorrer los 100 crímenes y agregar cada uno al grupo de características de incidentes
for lat, lng, in zip(df_incidentes.Latitude, df_incidentes.Longitude):
    incidentes.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=4, # tamaño de los marcadores
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# agregar texto emergente a cada marcador en el mapa
latitudes = list(df_incidentes.Latitude)
longitudes = list(df_incidentes.Longitude)
labels = list(df_incidentes.Tipo)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(mapa_gainesville)

# agregar incidentes al mapa
mapa_gainesville.add_child(incidentes)

Para cuando el mapa queda congestionado con tantos marcadores podemos utilizar la siguiente solución:

In [ ]:
# crear el mapa y mostrarlo
mapa_gainesville = folium.Map(location=[latitud_gainesville, longitud_gainesville], zoom_start=12)

# recorre los 100 crímenes y agrega cada uno al mapa
for lat, lng, label in zip(df_incidentes.Latitude, df_incidentes.Longitude, df_incidentes.Tipo):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, # tamaño de los marcadores circulares
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(mapa_gainesville)

# visualizar el mapa
mapa_gainesville

Otra foram de representarlos:

In [ ]:
from folium import plugins

# crear mapa
mapa_gainesville = folium.Map(location = [latitud_gainesville, longitud_gainesville], zoom_start = 12)

# crear una instancia de un objeto de grupo de marcas para los incidentes en el df
incidentes = plugins.MarkerCluster().add_to(mapa_gainesville)

# recorrer el marco de datos y agreguar cada punto de datos al grupo de marcas
for lat, lng, label, in zip(df_incidentes.Latitude, df_incidentes.Longitude, df_incidentes.Tipo):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidentes)

# visualizar el mapa
mapa_gainesville

Nota: Algunos ejemplos tienen como base material de cursos impartidos por IBM-

<div class="markdown-google-sans">

## Realizado por
</div>

<a href="https://uy.linkedin.com/in/noelia-mackinnon" target="_blank">Noelia Mackinnon</a>